Reorient and flip "flipped" fmri volumes
======

### Import everything you need to flip images

In [2]:
import nibabel as nib
import os.path as op
import os
from glob import glob

### Make a list of all images to be flipped
- remember to specify directory and image naming convention, if necessary

In [4]:
images = glob("/home/users/nsharif/ace_mount/ace_home/PING/niak/flip_func/*.nii.gz")
images

['/home/users/nsharif/ace_mount/ace_home/PING/niak/flip_func/P0270_v1_fmri_calib.nii.gz',
 '/home/users/nsharif/ace_mount/ace_home/PING/niak/flip_func/P0365_v1_fmri_rest.nii.gz',
 '/home/users/nsharif/ace_mount/ace_home/PING/niak/flip_func/P0011_v1_fmri_calib.nii.gz',
 '/home/users/nsharif/ace_mount/ace_home/PING/niak/flip_func/P0428_v1_fmri_rest.nii.gz',
 '/home/users/nsharif/ace_mount/ace_home/PING/niak/flip_func/P0276_v1_fmri_rest.nii.gz',
 '/home/users/nsharif/ace_mount/ace_home/PING/niak/flip_func/P0454_v1_fmri_calib.nii.gz',
 '/home/users/nsharif/ace_mount/ace_home/PING/niak/flip_func/P0206_v1_fmri_rest.nii.gz',
 '/home/users/nsharif/ace_mount/ace_home/PING/niak/flip_func/P1005_v1_fmri_calib.nii.gz',
 '/home/users/nsharif/ace_mount/ace_home/PING/niak/flip_func/P0340_v1_fmri_rest.nii.gz',
 '/home/users/nsharif/ace_mount/ace_home/PING/niak/flip_func/P0128_v1_fmri_calib.nii.gz',
 '/home/users/nsharif/ace_mount/ace_home/PING/niak/flip_func/P0604_v1_fmri_rest.nii.gz',
 '/home/users/ns

### Function to flip volumes
- will reorient whole volume and then flip every other volume so that all volumes are facing same direction as each other and as non-flipped fmri volumes that do not need to be corrected
- if whole volume does not need to be reoriented, comment out line 10

In [1]:
def reorient_flip_img_vols(image_in):
    # get basename for file
    base = image_in.split(".")[0]
    
    # load fmri volume + get vol data
    fmri_vol = nib.load(image_in)
    fmri_vol_data = fmri_vol.get_data()
    
    # next line reorients whole volume, comment out if unnecessary
    fmri_vol_data[:,:,:,:] = fmri_vol_data[:,::-1,:,:]
    
    # next line reorients every other volume to match preceeding volume
    fmri_vol_data[:,:,:,1::2] = fmri_vol_data[:,::-1,:,1::2]
    
    # save newly reoriented/flipped volume
    return nib.Nifti1Image(fmri_vol_data, fmri_vol.affine).to_filename('{}_reorient_flip.nii.gz'.format(base))
    

### Use that function!

In [7]:
for i in images:
    try:
        reorient_flip_img_vols(i)
    except:
        continue

Top up correction + application
======

### Now use topup.sh script
+ script will:
  1. topup calibration volume
  2. split fmri volume into AP & PA-only volumes
  3. apply topup to separated AP/PA volumes
  
### Remerge, if desired
+ if you want to remerge the toppedup volumes split by topup.sh, see below:

Remerge AP/PA volumes
======

### Import all the things necessary for remerging

In [2]:
import nibabel as ni
import numpy as np
import os.path as op
import os
from glob import glob

### Make list of images + subj IDs
+ remember to change directory and naming convention

In [ ]:
images = glob("/home/users/nsharif/ace_mount/ace_home/PING/niak/flip_func/P*_rest.nii.gz")

In [3]:
subjs = []
for img in images:
    subjs.append(img.split('/')[-1].split("_")[0])

NameError: name 'images' is not defined

### Remerge volumes using following loop

In [ ]:
for subj in subjs:
    #load all necessary images
    print('loading '+subj+'...')
    img_ap = ni.load('/home/users/nsharif/ace_mount/ace_home/PING/niak/flip_func/'+subj+'_v1_fmri_rest_AP_corrected.nii.gz').get_data()
    img_pa = ni.load('/home/users/nsharif/ace_mount/ace_home/PING/niak/flip_func/'+subj+'_v1_fmri_rest_PA_corrected.nii.gz').get_data()
    img_orig = ni.load('/home/users/nsharif/ace_mount/ace_home/PING/niak/flip_func/'+subj+'_v1_fmri_rest.nii.gz').get_data()
    aff = ni.load('/home/users/nsharif/ace_mount/ace_home/PING/niak/flip_func/'+subj+'_v1_fmri_rest.nii.gz').affine
    
    # check if image has uneven # of volumes
    if img_orig.shape[-1]%2 != 0:
        print(subj,'has uneven volumes:',img_orig.shape[-1])
        continue
    
    #make new scan as empty array based on shape of orig image
    newscan = np.zeros((64,64,47,img_orig.shape[-1]))

    # populate empty array with even-numbered volumes
    for scan,i in enumerate(list(range(0,img_orig.shape[-1],2))):
        newscan[:,:,:,i] = img_ap[:,:,:,scan]

    # populate empty array with odd-numbered volumes
    for scan,i in enumerate(list(range(1,img_orig.shape[-1]+1,2))):
        newscan[:,:,:,i] = img_pa[:,:,:,scan]
     
    # save newly merged volumes
    newimg = ni.Nifti1Image(newscan,aff)
    newimg.to_filename('/home/users/nsharif/ace_mount/ace_home/PING/niak/flip_func/'+subj+'_v1_fmri_remerge.nii.gz')